In [1]:
# Suppress FutureWarning
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
# Import required libraries and dependencies
import pandas as pd
#import hvplot.pandas
from pathlib import Path
#from sklearn.cluster import KMeans
#from sklearn.decomposition import PCA
#from sklearn.preprocessing import StandardScaler

#from sklearn.neighbors import NearestNeighbors
#from utils.fidatabase import TTR
#from utils.fidatabase import RTT

#import refinitiv.data as rd
#from refinitiv.data.content import symbol_conversion

#import pandas_market_calendars as mcal
import datetime as dt

import os
from pathlib import Path
#from io import BytesIO
#from zipfile import ZipFile
#from urllib.request import urlopen
#from urllib.parse import urlencode
#from string import ascii_uppercase as alc

import numpy as np
import re

#import requests 
from random import random

#from sklearn.neighbors import NearestNeighbors


In [2]:
first_ratetickers = pd.read_csv("first_ratetickers-list.csv", index_col="Ticker", parse_dates=True, keep_default_na=False) #,  infer_datetime_format=True)

first_ratetickers.columns = ['Name','start_date','end_date']
first_ratetickers["start_date"] = pd.to_datetime(first_ratetickers["start_date"], format='%Y-%m-%d').dt.tz_localize('America/New_York')
first_ratetickers["end_date"] = pd.to_datetime(first_ratetickers["end_date"], format='%Y-%m-%d').dt.tz_localize('America/New_York')


In [12]:
#
# Read comp files and run analysis
#

printed=False
swindow_list =  os.listdir("./comp_data")
ticker_window_list = {}
ticker_window_count = {}

regex = r"comps-(\d{4}\-\d{2}\-\d{2})\-(\d{4}\-\d{2}\-\d{2}).csv$"


for swindow in swindow_list:
    sdate = None
    edate = None
    
    matches = re.search(regex, swindow)
    if matches:
        sdate = pd.Timestamp(matches.group(1))
        edate = pd.Timestamp(matches.group(2))
       
        print(f"Processing window start_date: {sdate} end_date: {edate}")
    else:
        print(f"Skipping {swindow}")
        continue

    tmp_df = pd.read_csv(f"./comp_data/{swindow}", index_col=0, parse_dates=True, infer_datetime_format=True, keep_default_na=False)

    i=0

    for index, row in tmp_df.iterrows():
        i+=1
        
        if( index not in first_ratetickers.index ):
            print(f"{i} Skipping {index} not in first_ratetickers")
            continue
        
        x=row.tolist()
        if(printed==False):
            print(f"{i} {index} {x}")
            printed=True
            
        if( index not in ticker_window_count.keys() ):
            
            ticker_window_count[index] = 1
            ticker_window_list[index] = [x]
        else:
            ticker_window_count[index] += 1
            ticker_window_list[index].append(x)

    del tmp_df


print(f"Done!")


Processing window start_date: 2020-06-01 00:00:00 end_date: 2020-06-03 00:00:00
1 A ['GSK', 'IIM', 'ADX', 'NAD', 'RELX', 'JFR', 'TM', 'CHT', 'BCE', 'AAPL', 'NEA', 'BR', 'CII', 'UL', 'DHR', 'EBAY', 'TMO', 'QGEN', 'MYD', 'VZ', 1.607171498148266, 1.6368973195610608, 1.6445442237102432, 1.645434721482146, 1.649385816246182, 1.668197339227656, 1.6852610018579712, 1.6904158557617903, 1.6938592252989315, 1.698560837539544, 1.7039963354389984, 1.7173581471564048, 1.7242809467652995, 1.7325806006052702, 1.7431911970773566, 1.754520135167145, 1.75826976754037, 1.7712390347177982, 1.7791988532042182, 1.7839214152464131]
Processing window start_date: 2015-10-23 00:00:00 end_date: 2015-10-27 00:00:00
Processing window start_date: 2016-07-20 00:00:00 end_date: 2016-07-22 00:00:00
Processing window start_date: 2021-10-14 00:00:00 end_date: 2021-10-18 00:00:00
Processing window start_date: 2014-08-12 00:00:00 end_date: 2014-08-14 00:00:00
Processing window start_date: 2016-04-27 00:00:00 end_date: 201

In [159]:

def get_comps( stock_to_lookup, comps_per_frame=3, ignore_cut=False):

    stock_window_count = ticker_window_count[stock_to_lookup]
    stock_window_list = ticker_window_list[stock_to_lookup]

    stock_comps_count = {}
    stock_comps_distance = {}
    stock_comps_name = {}

    for cmp_list in stock_window_list:
        #row = tmp_df.loc[stock_to_lookup]
        if(len(cmp_list) < 35):
            print(f"Skipping {stock_to_lookup} not enough data")

        for i in range(0, comps_per_frame):
            ckey_value = cmp_list[i]
            dkey_value = cmp_list[i+20]

            if( ckey_value not in stock_comps_count.keys() ):
                stock_comps_count[ckey_value] = 1
                stock_comps_distance[ckey_value] = dkey_value
                stock_comps_name[ckey_value] = first_ratetickers.loc[ckey_value]["Name"]
            else:
                stock_comps_count[ckey_value] += 1
                stock_comps_distance[ckey_value] += dkey_value



        
    #stock_comps_count = {k: v for k, v in sorted(stock_comps_count.items(), reverse=True, key=lambda item: item[1])}

    rc_df = pd.DataFrame.from_dict(stock_comps_count, orient='index', columns=['Window'])
    rcd_df = pd.DataFrame.from_dict(stock_comps_distance, orient='index', columns=['TDistance'])
    rcn_df =  pd.DataFrame.from_dict(stock_comps_name, orient='index', columns=['Name'])

    comps_df = pd.concat([rcn_df,rc_df,rcd_df], axis=1)

    comps_df['TotalDistance'] = round(comps_df['TDistance'],2)
    comps_df['AvgDistance'] = round(comps_df['TDistance']/comps_df['Window'],2)
    comps_df['WindowPercent'] = round(comps_df['Window'] / stock_window_count * 100,2)

    comps_df = comps_df.sort_values(by=['Window'], ascending=False)

    #comps_cut = comps_df['Window'].min() + (comps_df['Window'].std()*.75)
    #comps_df.drop( comps_df[ comps_df['Window'] <= comps_cut ].index , inplace=True)


    #comps_cut = comps_df['Window'].mean() + comps_df['Window'].std()
    #comps_df.drop( comps_df[ comps_df['Window'] <= comps_cut ].index , inplace=True)

    return comps_df.iloc[:20]






In [206]:
comps_df = get_comps("MSFT", 4)
display(comps_df.head(21))

,Name,Window,TDistance,TotalDistance,AvgDistance,WindowPercent
AAPL,Apple,240,348.339017,348.34,1.45,10.97
ADP,Automatic Data Processing,234,336.613646,336.61,1.44,10.70
IBM,International Business Machines,223,327.268917,327.27,1.47,10.20
ORCL,Oracle,212,305.069874,305.07,1.44,9.69
INTU,Intuit,203,346.718829,346.72,1.71,9.28
ADBE,Adobe Systems Inc,195,270.465571,270.47,1.39,8.92
GOOG,Alphabet Inc Class C,193,276.453966,276.45,1.43,8.82
V,Visa,173,211.428547,211.43,1.22,7.91
GOOGL,Alphabet Inc Class A,166,246.477301,246.48,1.48,7.59
FISV,Fiserv Inc,142,224.275278,224.28,1.58,6.49


In [ ]:
ticker_list = pd.read_csv("full_stock_list_m.csv", index_col="Ticker", parse_dates=True, keep_default_na=False) #,  infer_datetime_format=True)

for index in ticker_list.index:
    output = Path(f"./comps_out/{index}.html")
    comps_df = get_comps(index, 4)
    comps_df.to_html(output)
    del comps_df

    break